In [1]:
import numpy as np
import pandas as pd
import re
import requests
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import plotly as py
import cufflinks as cf
from plotly.offline import iplot

py.offline.init_notebook_mode(connected=True)
cf.go_offline()

In [2]:
df=pd.read_excel('amazon_reviews.xlsx')
pd.set_option('max_colwidth',80)
df.head()

,rating,reviews,rate_pt
0,5.0 out of 5 stars,\n\n I would say this is the best available Mic in this price range. This m...,5
1,4.0 out of 5 stars,"\n\n Hi,So, after using this mic for about a month I am writing this review...",4
2,3.0 out of 5 stars,\n\n Product quality was good.. not bad.The build was also good. But one th...,3
3,1.0 out of 5 stars,\n\n WHEN RECORDING ON PHONE OR LApTOP THE quality was the same as you get ...,1
4,5.0 out of 5 stars,\n\n I will give my honest review on Boya M1. Read this full and i may help...,5


In [3]:
df.shape

(4839, 3)

In [4]:
# defining an empty data frame to amke a copy of data


In [5]:
# Converting Strings to Lowercase
df['reviews'] = [review.strip().lower() for review in df['reviews']]
df['reviews'][:5]

0    i would say this is the best available mic in this price range. this mic can...
1    hi,so, after using this mic for about a month i am writing this review.so, i...
2    product quality was good.. not bad.the build was also good. but one thing i ...
3    when recording on phone or laptop the quality was the same as you get from a...
4    i will give my honest review on boya m1. read this full and i may help you c...
Name: reviews, dtype: object

In [6]:
contractions = {
"ain't": "is not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"I'd": "I would",
"I'd've": "I would have",
"I'll": "I will",
"I'll've": "I will have",
"I'm": "I am",
"I've": "I have",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"

}

In [7]:
def expand_contractions(text):
    for word in text.split():
        if word.lower() in contractions:
            text = text.replace(word, contractions[word.lower()])
    return text

In [8]:
df['reviews'] = [expand_contractions(re.sub('’', "'", review)) for review in df['reviews']]
df['reviews'].head(5)

0    i would say this is the best available mic in this price range. this mic can...
1    hi,so, after using this mic for about a month i am writing this review.so, i...
2    product quality was good.. not bad.the build was also good. but one thing i ...
3    when recording on phone or laptop the quality was the same as you get from a...
4    i will give my honest review on boya m1. read this full and i may help you c...
Name: reviews, dtype: object

In [9]:
import unicodedata
def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    #https://docs.python.org/2/library/unicodedata.html
    return text

In [10]:
df['reviews'] = [remove_accented_chars(review) for review in df['reviews']]
df['reviews'].head(5)

0    i would say this is the best available mic in this price range. this mic can...
1    hi,so, after using this mic for about a month i am writing this review.so, i...
2    product quality was good.. not bad.the build was also good. but one thing i ...
3    when recording on phone or laptop the quality was the same as you get from a...
4    i will give my honest review on boya m1. read this full and i may help you c...
Name: reviews, dtype: object

In [11]:
# defining function to scrub special characters
def scrub_words(text):
    #Replace \xao characters in text
    text = re.sub('\xa0', ' ', text)
    
    #Replace non ascii / not words and digits
    text = re.sub("(\\W|\\d)",' ',text)
    
    #Replace new line characters and following text untill space
    text = re.sub('\n(\w*?)[\s]', '', text)
    
    #Remove html markup
    text = re.sub("<.*?>", ' ', text)
    
    #Remove extra spaces from the text
    text = re.sub("\s+", ' ', text)
    return text

In [12]:
pd.set_option('display.max_colwidth', -1)

In [13]:
df['reviews'] = [scrub_words(review) for review in df['reviews']]
df['reviews'].head(3)

0    i would say this is the best available mic in this price range this mic can be connected to your dslr smarthones laptops etc pros long wire sound quality is really good and richcons it will pick background noise if you are in a noisy environment but you can use recforge audio recorder app in your smartphone to remove all the noises apart from that it is all gold go for it                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

### <font> <font color="red"> Checking for NULL_VALUES

In [14]:
df.isna().sum()

rating     0
reviews    0
rate_pt    0
dtype: int64

### <font> <font color="red"> Removing stop words from df and applying lemmatization

In [15]:
# Importing spacy
import spacy


# Loading model
nlp = spacy.load('en_core_web_sm',disable=['parser', 'ner'])

# Lemmatization with stopwords removal
df['review_lemm']=df['reviews'].apply(lambda x: ' '.join([token.lemma_ for token in list(nlp(x)) if (token.is_stop==False)]))

In [16]:
df['review_lemm'].head(2)

0    well available mic price range mic connect dslr smarthone laptop etc pro long wire sound quality good richcon pick background noise noisy environment use recforge audio recorder app smartphone remove noise apart gold                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
1    hi mic month write review give genuine review mic beginner need budget mic pocket friendly give good output build quality mic piece compact handy use size sma

 ##  <font> <font color="red"> using VADER Sentiment Analysis on reviews to assign labels

In [17]:
# Import SentimentIntensityAnalyzer and create an sid object
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

In [20]:
df2=df.drop(['rating','rate_pt'],axis=1)
df2.head(2)

,reviews,review_lemm
0,i would say this is the best available mic in this price range this mic can be connected to your dslr smarthones laptops etc pros long wire sound quality is really good and richcons it will pick background noise if you are in a noisy environment but you can use recforge audio recorder app in your smartphone to remove all the noises apart from that it is all gold go for it,well available mic price range mic connect dslr smarthone laptop etc pro long wire sound quality good richcon pick background noise noisy environment use recforge audio recorder app smartphone remove noise apart gold
1,hi so after using this mic for about a month i am writing this review so i will be giving the most genuine review about this mic what a beginner needs a budget mic that is pocket friendly yet gives the best output built quality mic piece is very compact and and handy to use size is small so very convenient to carry anywhere you want cable wire is extraordinarily long meters however that is a great plus point long wire makes recording more easier it has a velcro for cable management so not a difficult task to keep the lengthy wire under control weighs around gms there are no batteries included there s a switch on off button while recording you need to switch to camera mode don t really know why its written camera sound quality my recordings have improved considerably with this mic and i am loving it just one complain i have it captures lot of air flow meaning i can hear lot of air of my mouth in the final output even if i record anything while holding the mic from a considerable distance i can hear my breathing sounds sound quality is loud enough and crisp it does capture my room s ceiling fan sound so i can hear that whoosh sound in the background when i am plugged into my earphones i record with the ac on and keep the fan switched off also it comes with a handy pouch to keep your mic safely yaayyy so finally best for production for a beginner like me because at this price it gives ample quality output,hi mic month write review give genuine review mic beginner need budget mic pocket friendly give good output build quality mic piece compact handy use size small convenient carry want cable wire extraordinarily long meter great plus point long wire make record easy velcro cable management difficult task lengthy wire control weigh gms battery include s switch button record need switch camera mode don t know write camera sound quality recording improve considerably mic love complain capture lot air flow meaning hear lot air mouth final output record hold mic considerable distance hear breathing sound sound quality loud crisp capture room s ceiling fan sound hear whoosh sound background plug earphone record ac fan switch come handy pouch mic safely yaayyy finally good production beginner like price give ample quality output


In [21]:
df2['score'] = df2['reviews'].apply(lambda reviews:sid.polarity_scores(reviews))
#df2.head(3)

In [22]:
df2['compound']  = df2['score'].apply(lambda score_dict: score_dict['compound'])
#df2.head(3)

In [23]:
df2['label'] = df2['compound'].apply(lambda c: 'pos' if c >0.05 else('neu'if  c<=0.050 and c>=-0.05 else 'neg'))

In [24]:
df2.head()

,reviews,review_lemm,score,compound,label
0,i would say this is the best available mic in this price range this mic can be connected to your dslr smarthones laptops etc pros long wire sound quality is really good and richcons it will pick background noise if you are in a noisy environment but you can use recforge audio recorder app in your smartphone to remove all the noises apart from that it is all gold go for it,well available mic price range mic connect dslr smarthone laptop etc pro long wire sound quality good richcon pick background noise noisy environment use recforge audio recorder app smartphone remove noise apart gold,"{'neg': 0.018, 'neu': 0.917, 'pos': 0.064, 'compound': 0.5182}",0.5182,pos
1,hi so after using this mic for about a month i am writing this review so i will be giving the most genuine review about this mic what a beginner needs a budget mic that is pocket friendly yet gives the best output built quality mic piece is very compact and and handy to use size is small so very convenient to carry anywhere you want cable wire is extraordinarily long meters however that is a great plus point long wire makes recording more easier it has a velcro for cable management so not a difficult task to keep the lengthy wire under control weighs around gms there are no batteries included there s a switch on off button while recording you need to switch to camera mode don t really know why its written camera sound quality my recordings have improved considerably with this mic and i am loving it just one complain i have it captures lot of air flow meaning i can hear lot of air of my mouth in the final output even if i record anything while holding the mic from a considerable distance i can hear my breathing sounds sound quality is loud enough and crisp it does capture my room s ceiling fan sound so i can hear that whoosh sound in the background when i am plugged into my earphones i record with the ac on and keep the fan switched off also it comes with a handy pouch to keep your mic safely yaayyy so finally best for production for a beginner like me because at this price it gives ample quality output,hi mic month write review give genuine review mic beginner need budget mic pocket friendly give good output build quality mic piece compact handy use size small convenient carry want cable wire extraordinarily long meter great plus point long wire make record easy velcro cable management difficult task lengthy wire control weigh gms battery include s switch button record need switch camera mode don t know write camera sound quality recording improve considerably mic love complain capture lot air flow meaning hear lot air mouth final output record hold mic considerable distance hear breathing sound sound quality loud crisp capture room s ceiling fan sound hear whoosh sound background plug earphone record ac fan switch come handy pouch mic safely yaayyy finally good production beginner like price give ample quality output,"{'neg': 0.017, 'neu': 0.832, 'pos': 0.152, 'compound': 0.9889}",0.9889,pos
2,product quality was good not bad the build was also good but one thing i want to say that please mention that it require a connector to connect with dslr most of entry level dslr or should provide a connector for dslr that is very useful and helpful for the customers because the connecter not available in any local market and when it buy through online shops some of it does not support the mic facility so we cannot get a proper product which support the mic and dslr i bought the product about i think weeks ago but still the product is in new condition that is it is in the box only because i did not get any connector suitable to connect with dslr and mic so kindly provide your own genuine connector for connecting with dslr it is a good and helpful for customers who is buying this product most of all customers are buying it for dslr so kindly do it for your valuable customers,product quality good bad build good thing want mention require 

In [25]:
df2.drop(['score','compound','review_lemm'],inplace=True,axis=1)
#df2.head()

In [26]:
df3=pd.DataFrame()
df3=df2.copy()
df3.to_excel("Amazon_labels_3.xlsx",index=False)